In [1]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('train_v2.csv', nrows = 50000)

In [3]:
df.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,6,1508176307
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,1,1508201613
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...","[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,1,1508169851
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...","[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,1,1508190552


In [4]:
json_columns = ['device', 'geoNetwork', 'totals', 'trafficSource']

In [5]:
for cols in json_columns:
    df_norm = pd.json_normalize([json.loads(val) for val in df[cols]]).fillna(0)
    df = pd.merge(df, df_norm, left_index=True, right_index=True)

In [6]:
df = df.drop(['socialEngagementType', 'browserVersion', 'browserSize', 'operatingSystemVersion',
              'mobileDeviceBranding', 'adwordsClickInfo.criteriaParameters', 'mobileDeviceModel', 
              'mobileInputSelector', 'mobileDeviceInfo', 'mobileDeviceMarketingName', 'flashVersion', 
              'language', 'screenColors', 'screenResolution', 'metro', 'city', 'adwordsClickInfo.isVideoAd',
              'cityId', 'networkDomain', 'latitude', 'longitude', 'networkLocation',
              'campaign', 'keyword', 'referralPath', 'adwordsClickInfo.gclId',
              'device', 'geoNetwork', 'totals', 'trafficSource', 'customDimensions', 'hits_x', 'visits'
             ], axis=1)

In [7]:
df.replace(['(not set)', 'not available in demo dataset', '(none)'], np.nan, inplace=True)
df['isTrueDirect'].replace(0, 'False', inplace=True)
df['adContent'].replace(0, np.nan, inplace=True)
df['adwordsClickInfo.adNetworkType'].replace(0, np.nan, inplace=True)
df['adwordsClickInfo.slot'].replace(0, np.nan, inplace=True)

In [8]:
df['pageviews'] = df['pageviews'].astype(int)
df['newVisits'] = df['newVisits'].astype(int)
df['bounces'] = df['bounces'].astype(int)
df['transactionRevenue'] = df['transactionRevenue'].astype(int)
df['fullVisitorId'] = df['fullVisitorId'].astype(str)
df['visitId'] = df['visitId'].astype(str)

In [9]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['weekday'] = df['date'].dt.weekday
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [10]:
df.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,browser,operatingSystem,isMobile,deviceCategory,...,medium,isTrueDirect,adContent,adwordsClickInfo.page,adwordsClickInfo.slot,adwordsClickInfo.adNetworkType,weekday,day,month,year
0,Organic Search,2017-10-16,3162355547410993243,1508198450,1,1508198450,Firefox,Windows,False,desktop,...,organic,False,NaN,0,NaN,NaN,0,16,10,2017
1,Referral,2017-10-16,8934116514970143966,1508176307,6,1508176307,Chrome,Chrome OS,False,desktop,...,referral,False,NaN,0,NaN,NaN,0,16,10,2017
2,Direct,2017-10-16,7992466427990357681,1508201613,1,1508201613,Chrome,Android,True,mobile,...,NaN,True,NaN,0,NaN,NaN,0,16,10,2017
3,Organic Search,2017-10-16,9075655783635761930,1508169851,1,1508169851,Chrome,Windows,False,desktop,...,organic,False,NaN,0,NaN,NaN,0,16,10,2017
4,Organic Search,2017-10-16,6960673291025684308,1508190552,1,1508190552,Chrome,Windows,False,desktop,...,organic,False,NaN,0,NaN,NaN,0,16,10,2017


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   channelGrouping                 50000 non-null  object        
 1   date                            50000 non-null  datetime64[ns]
 2   fullVisitorId                   50000 non-null  object        
 3   visitId                         50000 non-null  object        
 4   visitNumber                     50000 non-null  int64         
 5   visitStartTime                  50000 non-null  int64         
 6   browser                         50000 non-null  object        
 7   operatingSystem                 49663 non-null  object        
 8   isMobile                        50000 non-null  bool          
 9   deviceCategory                  50000 non-null  object        
 10  continent                       49927 non-null  object        
 11  su

In [12]:
df.describe()

,visitNumber,visitStartTime,pageviews,bounces,newVisits,transactionRevenue,weekday,day,month,year
count,50000.00000,5.000000e+04,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000
mean,2.27366,1.495045e+09,3.631100,0.507960,0.769460,1.203764e+06,3.351880,14.162240,7.165600,2016.823760
std,8.35306,1.614779e+07,6.310063,0.499942,0.421183,3.293068e+07,1.482479,7.203949,3.998852,0.661081
min,1.00000,1.470899e+09,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2.000000,1.000000,2016.000000
25%,1.00000,1.479457e+09,1.000000,0.000000,1.000000,0.000000e+00,2.000000,10.000000,3.000000,2016.000000
50%,1.00000,1.497370e+09,1.000000,1.000000,1.000000,0.000000e+00,4.000000,13.000000,9.000000,2017.000000
75%,1.00000,1.510405e+09,3.000000,1.000000,1.000000,0.000000e+00,4.000000,18.000000,11.000000,2017.000000
max,401.00000,1.523862e+09,197.000000,1.000000,1.000000,5.498000e+09,6.000000,30.000000,11.000000,2018.000000
